# [음성 분류 경진대회](https://dacon.io/competitions/official/235905/overview/description)

In [2]:
import librosa
import librosa.display as dsp
from IPython.display import Audio

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [4]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(929)